<a href="https://colab.research.google.com/github/jasreman8/LLMs-For-RAGs-II/blob/main/RAG_Evaluation_using_DeepEval_with_RAG_Triad_and_RAGAS_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Objectives

- Illustrate RAG evaluation using the `deepeval` package to evaluate RAG outputs using:
    - RAG Triad metrics
    - Ragas metrics


# Setup

- deepeval provides us with evaluation metrics.
- chromadb is an open source vector database optimized for AI-native applications that stores text embeddings with associated metadata.
- RAGAS --> RAG Assessment Score library uses OpenAI models to score generated answers

!pip install -q openai==1.66.3 \
                tiktoken==0.9.0 \
                langchain==0.3.20 \
                langchain-chroma==0.2.2 \
                langchain-openai==0.3.9 \
                chromadb==0.6.3 \
                deepeval==3.6.9

In [1]:
import os, time
import chromadb

from openai import OpenAI # OpenAI client

from deepeval import evaluate
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.metrics import ContextualPrecisionMetric, ContextualRecallMetric

from google.colab import userdata

In [2]:
openai_api_key = userdata.get('my_api_key')

os.environ['OPENAI_API_KEY'] = userdata.get('my_api_key')

In [3]:
!unzip tesla_db.zip

Archive:  tesla_db.zip
replace tesla_db/chroma.sqlite3? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: tesla_db/chroma.sqlite3  
   creating: tesla_db/e5401e39-e25e-4be0-8e53-57cb38e8c579/
  inflating: tesla_db/e5401e39-e25e-4be0-8e53-57cb38e8c579/header.bin  
  inflating: tesla_db/e5401e39-e25e-4be0-8e53-57cb38e8c579/link_lists.bin  
  inflating: tesla_db/e5401e39-e25e-4be0-8e53-57cb38e8c579/length.bin  
  inflating: tesla_db/e5401e39-e25e-4be0-8e53-57cb38e8c579/index_metadata.pickle  
  inflating: tesla_db/e5401e39-e25e-4be0-8e53-57cb38e8c579/data_level0.bin  


# Assembling Test Cases

An important first steps to evaluate RAG responses is to assemble test cases.

A typical test case consists of:
- query representative of expected user questions when the application goes live
- response generated by the RAG application
- (optional) human baseline answer for the same query using the same data sources

While optional, having a small subset of human baselines answers (gold answers) improves the robustness of the evaluation process.

## Composing test case response

Let us begin by setting up the full RAG pipeline that takes in user queries and uses retrieved context from the vector database to answer this question.

In [4]:
qna_system_message = """
You are an assistant to a financial services firm who answers user queries on annual reports.
User input will have the context required by you to answer user queries.
This context will be delimited by: <Context> and </Context>.
The context contains references to specific portions of a document relevant to the user query.

User queries will be delimited by: <Question> and </Question>.

Please answer user queries only using the context provided in the input.
Do not mention anything about the context in your final answer. Your response should only contain the answer to the question.

If the answer is not found in the context, respond "I don't know".
"""

qna_user_message_template = """
<Context>
Here are some documents that are relevant to the question mentioned below.
{context}
</Context>

<Question>
{question}
</Question>
"""

def answer(user_query: str) -> str:
    """Answers user queries using context retrieved from a vector database.

    Retrieves relevant document chunks from a vector database based on the user query,
    formats the context and query into a prompt, and sends it to a large language model
    for answer generation.

    Args:
        user_query: The user's query.

    Returns:
        A tuple containing the generated answer and the list of retrieved context.
    """
    client = OpenAI(
        api_key=openai_api_key,
    )

    model_name = 'gpt-4o-mini'

    embedding_model = OpenAIEmbeddings(
        api_key=openai_api_key,
        model='text-embedding-3-small'
    )

    chromadb_client = chromadb.PersistentClient(
        path="./tesla_db"
    )

    tesla_10k_collection = 'tesla-10k-2019-to-2023'

    vectorstore_persisted = Chroma(
        collection_name=tesla_10k_collection,
        collection_metadata={"hnsw:space": "cosine"},
        embedding_function=embedding_model,
        client=chromadb_client,
        persist_directory="./tesla_db"
    )

    retriever = vectorstore_persisted.as_retriever(
        search_type='similarity',
        search_kwargs={'k': 5}
    )

    relevant_document_chunks = retriever.invoke(user_query)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = "\n---\n".join(context_list)

    prompt = [
        {'role': 'developer', 'content': qna_system_message},
        {'role': 'user', 'content': qna_user_message_template.format(
            context=context_for_query,
            question=user_query
            )
        }
    ]

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=prompt,
            temperature=0
        )

        prediction = response.choices[0].message.content.strip()
    except Exception as e:
        prediction = f'Sorry, I encountered the following error: \n {e}'

    return prediction, context_list

There are two use case scenarios to consider for each query in the test set:
- when baseline human responses (gold responses) are available
- when baseline human responses are not available

It is always preferable to have gold responses for a sample of test cases.

## Test cases with human baseline

Let us assemble a test case when golden output is available. This is done using the `LLMTestCase` abstraction from `deepeval`.

In [5]:
test_query = "What was the total revenue of the company in 2022?"

In [6]:
golden_output = '$81.46 billion'

In [7]:
output, retrieved_context = answer(test_query)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


In [8]:
output

'In 2022, the total revenues of the company were $81.46 billion.'

In [9]:
test_case_with_golden_output = LLMTestCase(
    input=test_query,
    expected_output=golden_output,
    actual_output=output,
    retrieval_context=retrieved_context
)

## Test cases without human baseline


The `LLMTestCase` abstraction also works when there is no expected output. In this case, the output is evaluated with an LLM-as-a-judge approach using the metric definition.

In [10]:
test_query = "What was the total revenue of the company in 2022?"

In [11]:
output, retrieved_context = answer(test_query)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [12]:
output

'In 2022, the total revenues of the company were $81.46 billion.'

In [13]:
test_case_without_golden_output = LLMTestCase(
    input=test_query,
    actual_output=output,
    retrieval_context=retrieved_context
)

# Evaluation - RAG Triad

The `deepeval` framework offers ready-to-use implementations of the three metrics comprising the RAG Triad: answer relevance, faithfulness, and contextual relevance.

An essential configuration parameter in this framework is the threshold, which defines the minimum acceptable standard for an answer to pass evaluation against the chosen metric. Lowering this threshold decreases the test's stringency, which can, in some cases, be advantageous. Excessively stringent thresholds may result in a high failure rate, potentially hindering the progression of the application towards deployment. Balancing this parameter is therefore critical to ensure both rigorous evaluation and practical viability.

We recommended to pick a reasonable threshold before starting the evaluation and stick with it as we change the parameters of the RAG system (e.g., chunk size, embedding model).

In [14]:
answer_relevancy = AnswerRelevancyMetric(
    threshold=0.7,
    model='gpt-4o',
    include_reason=True
)

faithfulness = FaithfulnessMetric(
    threshold=0.7,
    model='gpt-4o',
    include_reason=True
)

contextual_relevancy = ContextualRelevancyMetric(
    threshold=0.7,
    model='gpt-4o',
    include_reason=True
)

## Single Test Case

With the metrics in place we can now evaluate the performance of the RAG system against the test case using the `evaluate` function.

In [15]:
results = evaluate(
    test_cases=[test_case_with_golden_output],
    metrics=[answer_relevancy, faithfulness, contextual_relevancy]
)

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases




Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because the response perfectly addressed the question about the company's total revenue in 2022 without any irrelevant information. Great job on staying focused and precise!, error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because there are no contradictions between the actual output and the retrieval context. Everything aligns perfectly, showcasing a high level of accuracy and consistency. Keep up the great work!, error: None)
  - ❌ Contextual Relevancy (score: 0.03571428571428571, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 0.04 because the retrieval context predominantly contains information unrelated to the company's total revenue in 2022, such as data from other years and different financial metrics. However, one relevant statement p

⚠ WARNING: No hyperparameters logged.
» ]8;id=981706;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 5.78s | token cost: 0.05649 USD)
» Test Results (1 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 1

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

As the above output indicates, the RAG system passes answer relevancey and faithfulness but fails the contextual relevancy test. This is because not all the retrieved documents are relevant to the query. The next step in this case would be to improve the retrieval mechanism (e.g., change chunk size, $k$ or the embedding model).

We can also repeat the evaluation for the situation where the gold output is not available.

In [16]:
results = evaluate(
    test_cases=[test_case_without_golden_output],
    metrics=[answer_relevancy, faithfulness, contextual_relevancy]
)

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases




Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because the response perfectly addressed the question about the company's total revenue in 2022 without any irrelevant information. Great job on staying focused and precise!, error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because there are no contradictions between the actual output and the retrieval context. Everything aligns perfectly, showcasing a high level of accuracy and consistency. Great job!, error: None)
  - ❌ Contextual Relevancy (score: 0.04, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 0.04 because the retrieval context primarily contains information unrelated to the 2022 revenue, such as data from other years and different financial metrics. However, one relevant statement mentions, 'In 2022, we recognized total revenues of 

⚠ WARNING: No hyperparameters logged.
» ]8;id=741142;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 6.45s | token cost: 0.05422250000000001 USD)
» Test Results (1 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 1

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

## Batch Evaluation

When moving from single test evaluation to evaluating a batch of queries, we need to create an evaluation dataset. This involved looping over a sample of test queries, baseline answers (gold outputs) and adding test cases to the dataset like so:

In [17]:
test_queries = [
    "What was the total revenue of the company in 2022?",
    "What was the company's debt level in 2023?",
    "Present 3 key highlights of the Management Discussion and Analysis section of the 2022 report in 50 words."
]

In [18]:
golden_outputs = [
    "$81.46 billion",
    "$2,061 million ($1,016 million in recourse debt and $1,029 million in non-recourse debt)",
    """
    In 2022, Tesla produced 1.37 million vehicles, recognized revenues of $81.46 billion, and achieved a net income of $12.56 billion.
    The company focused on increasing production capacity, improving battery technologies, and expanding its energy storage and solar energy systems.
    """
]

In [19]:
dataset = EvaluationDataset()

In [20]:
for gold_query, gold_output in zip(test_queries, golden_outputs):
    actual_output, retrieved_context = answer(gold_query)

    dataset.add_test_case(
        LLMTestCase(
            input=gold_query,
            expected_output=gold_output,
            actual_output=actual_output,
            retrieval_context=retrieved_context
        )
    )

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Once the dataset is assembled, the evaluation can be executed on the dataset using the RAG Triad metrics we saw earlier.

In [21]:
for tc in dataset.test_cases:
  for metric in [answer_relevancy, faithfulness, contextual_relevancy]:
    evaluate(test_cases=[tc],metrics=[metric])
    time.sleep(2)
  time.sleep(2)

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases




Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because the answer perfectly addressed the question about the company's total revenue in 2022 without any irrelevant information. Great job on staying focused and precise!, error: None)

For test case:

  - input: What was the total revenue of the company in 2022?
  - actual output: The total revenue of the company in 2022 was $81.46 billion.
  - expected output: $81.46 billion
  - context: None
  - retrieval context: ['systems.\nIn\t2020,\twe\trecognized\ttotal\trevenues\tof\t$31.54\tbillion,\trepresenting\tan\tincrease\tof\t$6.96\tbillion\tcompared\tto\tthe\tprior\tyear.\tWe\tcontinue\tto\tramp\nproduction,\tbuild\tnew\tmanufacturing\tcapacity\tand\texpand\tour\toperations\tto\tenable\tincreased\tdeliveries\tand\tdeployments\tof\tour\tproducts\tand\tfurther\trevenue\ngrowth.\nIn\t2020,\tour\tnet\tincome\tattributable\tto\tcommon\tstockholders\twas\

⚠ WARNING: No hyperparameters logged.
» ]8;id=680172;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases




Metrics Summary

  - ❌ Contextual Relevancy (score: 0.043478260869565216, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 0.04 because the retrieval context primarily contains irrelevant information, such as references to other years and sections, while only one statement, 'In 2022, we produced 1,369,611 consumer vehicles and delivered 1,313,851 consumer vehicles, despite ongoing supply chain and logistics challenges and factory shutdowns,' is relevant to the input request for 2022 highlights., error: None)

For test case:

  - input: Present 3 key highlights of the Management Discussion and Analysis section of the 2022 report in 50 words.
  - actual output: In 2022, the company produced 1,369,611 consumer vehicles and delivered 1,313,851 despite supply chain challenges. The focus was on increasing production capacity and delivery capabilities. Additionally, the company emphasized its commitment to sustainable energy and expanding services related to art

⚠ WARNING: No hyperparameters logged.
» ]8;id=398564;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 5.63s | token cost: 0.027745000000000002 USD)
» Test Results (1 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 1

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

# Evaluation - Ragas

Evaluation using the Ragas metrics can be done using the same set of steps as the RAG Triad. These metrics are available out-of-the-box in `deepeval` and can be instantiated for batch evaluation as we did with RAG Triad metrics.

In [22]:
answer_relevancy = AnswerRelevancyMetric(
    threshold=0.7,
    model='gpt-4o',
    include_reason=True
)

faithfulness = FaithfulnessMetric(
    threshold=0.7,
    model='gpt-4o',
    include_reason=True
)

contextual_precision = ContextualPrecisionMetric(
    threshold=0.7,
    model='gpt-4o',
    include_reason=True
)

contextual_recall = ContextualRecallMetric(
    threshold=0.7,
    model='gpt-4o',
    include_reason=True
)

In [23]:
for tc in dataset.test_cases:
    for metric in [answer_relevancy, faithfulness, contextual_precision, contextual_recall]:
        evaluate(test_cases=[tc], metrics=[metric])
        time.sleep(2)
    time.sleep(2)

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases




Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because the response perfectly addressed the question about the company's total revenue in 2022 without any irrelevant information. Great job on staying focused and precise!, error: None)

For test case:

  - input: What was the total revenue of the company in 2022?
  - actual output: The total revenue of the company in 2022 was $81.46 billion.
  - expected output: $81.46 billion
  - context: None
  - retrieval context: ['systems.\nIn\t2020,\twe\trecognized\ttotal\trevenues\tof\t$31.54\tbillion,\trepresenting\tan\tincrease\tof\t$6.96\tbillion\tcompared\tto\tthe\tprior\tyear.\tWe\tcontinue\tto\tramp\nproduction,\tbuild\tnew\tmanufacturing\tcapacity\tand\texpand\tour\toperations\tto\tenable\tincreased\tdeliveries\tand\tdeployments\tof\tour\tproducts\tand\tfurther\trevenue\ngrowth.\nIn\t2020,\tour\tnet\tincome\tattributable\tto\tcommon\tstockholders\twa

⚠ WARNING: No hyperparameters logged.
» ]8;id=534754;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases




Metrics Summary

  - ✅ Contextual Recall (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because every sentence in the expected output is directly supported by the information in the node(s) in the retrieval context, ensuring complete alignment and accuracy., error: None)

For test case:

  - input: Present 3 key highlights of the Management Discussion and Analysis section of the 2022 report in 50 words.
  - actual output: In 2022, the company produced 1,369,611 consumer vehicles and delivered 1,313,851 despite supply chain challenges. The focus was on increasing production capacity and delivery capabilities. Additionally, the company emphasized its commitment to sustainable energy and expanding services related to artificial intelligence, robotics, and automation.
  - expected output: 
    In 2022, Tesla produced 1.37 million vehicles, recognized revenues of $81.46 billion, and achieved a net income of $12.56 billion.
    The company f

⚠ WARNING: No hyperparameters logged.
» ]8;id=193928;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 2.89s | token cost: 0.008667500000000002 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.